# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Walter  ->  F. Walter  |  ['F. Walter']
T. K. Henning  ->  T. K. Henning  |  ['T. K. Henning']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
L. Eisert  ->  L. Eisert  |  ['L. Eisert']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
J. Liu  ->  J. Liu  |  ['J. Liu']
R. Zhang  ->  R. Zhang  |  ['R. Zhang']
K. Lee  ->  K. Lee  |  ['K. Lee']
S. Li  ->  S. Li  |  ['S. Li']
J. Li  ->  J. Li  |  ['J. Li']


D. Semenov  ->  D. Semenov  |  ['D. Semenov']
S. Li  ->  S. Li  |  ['S. Li']
Arxiv has 85 new papers today
          11 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/11 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2407.00153
extracting tarball to tmp_2407.00153... done.


list index out of range
Retrieving document from  https://arxiv.org/e-print/2407.00165


extracting tarball to tmp_2407.00165...

 done.
Retrieving document from  https://arxiv.org/e-print/2407.00166
extracting tarball to tmp_2407.00166...

...

extracting tarball to tmp_2407.01413...

 done.
Retrieving document from  https://arxiv.org/e-print/2407.01438
extracting tarball to tmp_2407.01438...

/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2093/1211882699.py:51: LatexWarning: 2407.01413 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2093/1211882699.py:51: LatexWarning: 2407.01438 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.00153-b31b1b.svg)](https://arxiv.org/abs/2407.00153) | **MIDIS. Near-infrared rest-frame morphology of massive galaxies at $3<z<5.5$ in the Hubble eXtreme Deep Field**  |
|| L. Costantin, et al. -- incl., <mark>F. Walter</mark>, <mark>T. K. Henning</mark> |
|*Appeared on*| *2024-07-03*|
|*Comments*| *Submitted to A&A. 7 pages, 4 figures. Comments welcome*|
|**Abstract**|            Thanks to decades of observations using HST, the structure of galaxies at redshift $z>2$ has been widely studied in the rest-frame ultraviolet regime, which traces recent star formation from young stellar populations. But, we still have little information about the spatial distribution of the older, more evolved, stellar populations, constrained by the rest-frame infrared portion of galaxies' spectral energy distribution. We present the morphological characterization of a sample of 21 massive galaxies ($\log(M_{\star}/M_{\odot})>9.5$) at redshift $3<z<5.5$. These galaxies are observed as part of the GTO program MIDIS with the Mid-Infrared Instrument (MIRI) onboard JWST. The deep MIRI 5.6~$\mu$m imaging allows us to characterize for the first time the rest-frame near-infrared structure of galaxies beyond cosmic noon, at higher redshifts than possible with NIRCam, tracing their older stellar populations. We derive the galaxies' non-parametric morphology and model the galaxies' light distribution with a Sérsic component. We find that at $z>3$ massive galaxies show a smooth distribution of their rest-infrared light, strongly supporting the increasing number of regular disk galaxies already in place at early epochs. On the contrary, the ultraviolet structure obtained from HST observations is generally more irregular, catching the most recent episodes of star formation. Importantly, we find a segregation of morphologies across cosmic time, having massive galaxies at redshift $z>4$ later-type morphologies compared to $z\sim3$ galaxies. These findings suggest a transition phase in galaxy assembly and central mass build up already taking place at $z\sim3-4$. MIRI provides unique information about the structure of the mature stellar population of high-redshift galaxies, unveiling that massive galaxies beyond cosmic noon are prevalently compact disk galaxies with smooth mass distribution.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.00166-b31b1b.svg)](https://arxiv.org/abs/2407.00166) | **Constraints on the in-situ and ex-situ stellar masses in nearby galaxies with Artificial Intelligence**  |
|| E. Angeloudi, et al. -- incl., <mark>L. Eisert</mark>, <mark>A. Pillepich</mark> |
|*Appeared on*| *2024-07-03*|
|*Comments*| *16 pages, 7 Figures. Accepted for publication*|
|**Abstract**|            The hierarchical model of galaxy evolution suggests that the impact of mergers is substantial on the intricate processes that drive stellar assembly within a galaxy. However, accurately measuring the contribution of accretion to a galaxy's total stellar mass and its balance with in-situ star formation poses a persistent challenge, as it is neither directly observable nor easily inferred from observational properties. Here, we present theory-motivated predictions for the fraction of stellar mass originating from mergers in a statistically significant sample of nearby galaxies, using data from MaNGA. Employing a robust machine learning model trained on mock MaNGA analogs (MaNGIA) in turn obtained from a cosmological simulation (TNG50), we unveil that in-situ stellar mass dominates almost across the entire stellar mass spectrum (1e9Msun < M* < 1e12Msun). Only in more massive galaxies (M* > 1e11Msun) does accreted mass become a substantial contributor, reaching up to 35-40% of the total stellar mass. Notably, the ex-situ stellar mass in the nearby universe exhibits significant dependence on galaxy characteristics, with higher accreted fractions favored by elliptical, quenched galaxies and slow rotators, as well as galaxies at the center of more massive dark matter halos.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.00171-b31b1b.svg)](https://arxiv.org/abs/2407.00171) | **The PARADIGM project I: How early merger histories shape the present-day sizes of Milky-Way-mass galaxies**  |
|| G. D. Joshi, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2024-07-03*|
|*Comments*| *Submitted to MNRAS. 22 pages. Comments are welcome*|
|**Abstract**|            The way in which mergers affect galaxy formation depends on both feedback processes, and on the geometry and strength of the mergers themselves. We introduce the PARADIGM project, where we study the response of a simulated Milky-Way-mass galaxy forming in a cosmological setting to differing merger histories, using genetically modified initial conditions. Each initial condition is simulated with the VINTERGATAN and IllustrisTNG codes. While VINTERGATAN has been developed with an emphasis on resolving the cold interstellar medium, IllustrisTNG uses a subgrid two-phase model and consequently scales to large volume simulations, making them ideal to examine complementary views on how merger histories and feedback interact. Our genetic modifications alter the mass ratio of an important $z \approx 2$ merger while maintaining the halo's $z=0$ mass. Whether simulated with VINTERGATAN or IllustrisTNG, smaller mass ratios for this early merger result in larger galaxies at $z=0$, due to a greater build up of a kinematically cold disc. We conclude that such broad trends are robustly reproducible; however, the normalization of the resulting stellar sizes is substantially different in the two codes (ranging between $0.5-1.7\ \rm{kpc}$ for VINTERGATAN but $1.3-7.0\ \rm{kpc}$ for IllustrisTNG). The VINTERGATAN galaxies systematically form stars earlier, leading to a larger bulge component. Despite the difference in size normalization, both simulation suites lie on the observed size-mass relation for their respective morphological types. In light of these results, we discuss the interplay between internal processes and large scale gravitational interactions and gas accretion, and how the two galaxy models converge on similar emergent trends but along different evolutionary pathways.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.01063-b31b1b.svg)](https://arxiv.org/abs/2407.01063) | **Direct observational evidence of multi-epoch massive star formation in G24.47+0.49**  |
|| A. Saha, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2024-07-03*|
|*Comments*| *18 pages, 7 figures, accepted for publication in The Astrophysical Journal Letters*|
|**Abstract**|            Using new continuum and molecular line data from the ALMA Three-millimeter Observations of Massive Star-forming Regions (ATOMS) survey and archival VLA, 4.86 GHz data, we present direct observational evidence of hierarchical triggering relating three epochs of massive star formation in a ring-like H II region, G24.47+0.49. We find from radio flux analysis that it is excited by a massive star(s) of spectral type O8.5V-O8V from the first epoch of star formation. The swept-up ionized ring structure shows evidence of secondary collapse, and within this ring a burst of massive star formation is observed in different evolutionary phases, which constitutes the second epoch. ATOMS spectral line (e.g., HCO$^+$(1-0)) observations reveal an outer concentric molecular gas ring expanding at a velocity of $\sim$ 9 $\rm km\,s^{-1}$, constituting the direct and unambiguous detection of an expanding molecular ring. It harbors twelve dense molecular cores with surface mass density greater than 0.05 $\rm g\,cm^{-2}$, a threshold typical of massive star formation. Half of them are found to be subvirial, and thus in gravitational collapse, making them third epoch of potential massive star-forming sites.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.00977-b31b1b.svg)](https://arxiv.org/abs/2407.00977) | **Ongoing and fossil large-scale outflows detected in a high-redshift radio galaxy: [C II] observations of TN J0924$-$2201 at $z=5.174$**  |
|| <mark>K. Lee</mark>, et al. |
|*Appeared on*| *2024-07-03*|
|*Comments*| *16 pages, 11 figures, accepted for publication in ApJ*|
|**Abstract**|            We present Atacama Large Millimeter/submillimeter Array observations of the [C II] 158 $\mu$m line and the underlying continuum emission of TN J0924$-$2201, which is one of the most distant known radio galaxies at $z>5$. The [C II] line and 1-mm continuum emission are detected at the host galaxy. The systemic redshift derived from the [C II] line is $z_{\rm [C II]}=5.1736\pm0.0002$, indicating that the Ly$\alpha$ line is redshifted by a velocity of $1035\pm10$ km s$^{-1}$, marking the largest velocity offset between the [C II] and Ly$\alpha$ lines recorded at $z>5$ to date. In the central region of the host galaxy, we identified a redshifted substructure of [C II] with a velocity of $702\pm17$ km s$^{-1}$, which is close to the CIV line with a velocity of $500\pm10$ km s$^{-1}$. The position and the velocity offsets align with a model of an outflowing shell structure, consistent with the large velocity offset of Ly$\alpha$. The non-detection of [C II] and dust emission from the three CO(1--0)-detected companions indicates their different nature compared to dwarf galaxies based on the photodissociation region model. Given their large velocity of $\sim1500$ km s$^{-1}$, outflowing molecular clouds induced by the AGN is the most plausible interpretation, and they may exceed the escape velocity of a $10^{13}\,M_{\odot}$ halo. These results suggest that TN J0924$-$2201, with the ongoing and fossil large-scale outflows, is in a distinctive phase of removing molecular gas from a central massive galaxy in an overdense region in the early universe. A dusty HI absorber at the host galaxy is an alternative interpretation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.01118-b31b1b.svg)](https://arxiv.org/abs/2407.01118) | **Preliminary results of sky brightness measurements in near-infrared at Lenghu, China**  |
|| <mark>J. Li</mark>, B. Ma, Z. Dong, H. Zhang |
|*Appeared on*| *2024-07-03*|
|*Comments*| *8 pages, 8 figures, presented at 2024 SPIE Astronomical Telescopes + Instrumentation conference*|
|**Abstract**|            Low sky brightness is crucial for ground-based astronomical observations, because it limits the observational capability to detect fainter sources. Lenghu, located on the Tibetan Plateau in China, has been identified as an high-quality astronomical site in China, including dark sky in optical band. In this work, we will report the preliminary results of near-infrared sky brightness measurements at Lenghu. Utilizing a wide-field small telescope equipped with an InGaAs camera, we have been conducting long-term monitoring of near-infrared sky brightness in the J and H' bands, respectively, since January 2024. For each image, photometry and astrometry were performed, then sky background was calibrated by standard stars from the 2MASS catalog. This report includes preliminary results on the sky brightness at zenith in the J and H' bands, as well as their variations with solar elevation at Lenghu. Our initial results indicate that the near-infrared sky brightness at Lenghu is comparable to that of other world-class sites, and long-term monitoring will be continued.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.00165-b31b1b.svg)](https://arxiv.org/abs/2407.00165) | **High-energy spectra of LTT 1445A and GJ 486 reveal flares and activity**  |
|| H. Diamond-Lowe, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2024-07-03*|
|*Comments*| *22 pages, accepted to A&A*|
|**Abstract**|            The high-energy radiative output, from the X-ray to the ultraviolet, of exoplanet host stars drives photochemical reactions and mass loss in the upper regions of planetary atmospheres. In order to place constraints on the atmospheric properties of the three closest terrestrial exoplanets transiting M dwarfs, we observe the high-energy spectra of the host stars LTT1445A and GJ486 in the X-ray with XMM-Newton and Chandra and in the ultraviolet with HST/COS and STIS. We combine these observations with estimates of extreme ultraviolet flux, reconstructions of the Ly-a lines, and stellar models at optical and infrared wavelengths to produce panchromatic spectra from 1A--20um for each star. While LTT1445Ab, LTT1445Ac, and GJ486b do not possess primordial hydrogen-dominated atmospheres, we calculate that they are able to retain pure CO2 atmospheres if starting with 10, 15, and 50% of Earth's total CO2 budget, respectively, in the presence of their host stars' stellar wind. We use age-activity relationships to place lower limits of 2.2 and 6.6 Gyr on the ages of the host stars LTT1445A and GJ486. Despite both LTT1445A and GJ486 appearing inactive at optical wavelengths, we detect flares at ultraviolet and X-ray wavelengths for both stars. In particular, GJ486 exhibits two flares with absolute energies of 10^29.5 and 10^30.1 erg (equivalent durations of 4357+/-96 and 19724+/-169 s) occurring three hours apart, captured with HST/COS G130M. Based on the timing of the observations, we suggest that these high-energy flares are related and indicative of heightened flaring activity that lasts for a period of days, but our interpretations are limited by sparse time-sampling. Consistent high-energy monitoring is needed to determine the duration and extent of high-energy activity on individual M dwarfs, as well as the population as a whole.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.01413-b31b1b.svg)](https://arxiv.org/abs/2407.01413) | **AtLAST Science Overview Report**  |
|| M. Booth, et al. -- incl., <mark>D. Semenov</mark> |
|*Appeared on*| *2024-07-03*|
|*Comments*| *47 pages, 12 figures. For further details on AtLAST see this https URL*|
|**Abstract**|            Submillimeter and millimeter wavelengths provide a unique view of the Universe, from the gas and dust that fills and surrounds galaxies to the chromosphere of our own Sun. Current single-dish facilities have presented a tantalising view of the brightest (sub-)mm sources, and interferometers have provided the exquisite resolution necessary to analyse the details in small fields, but there are still many open questions that cannot be answered with current facilities. In this report we summarise the science that is guiding the design of the Atacama Large Aperture Submillimeter Telescope (AtLAST). We demonstrate how tranformational advances in topics including star formation in high redshift galaxies, the diffuse circumgalactic medium, Galactic ecology, cometary compositions and solar flares motivate the need for a 50m, single-dish telescope with a 1-2 degree field of view and a new generation of highly multiplexed continuum and spectral cameras. AtLAST will have the resolution to drastically lower the confusion limit compared to current single-dish facilities, whilst also being able to rapidly map large areas of the sky and detect extended, diffuse structures. Its high sensitivity and large field of view will open up the field of submillimeter transient science by increasing the probability of serendipitous detections. Finally, the science cases listed here motivate the need for a highly flexible operations model capable of short observations of individual targets, large surveys, monitoring programmes, target of opportunity observations and coordinated observations with other observatories. AtLAST aims to be a sustainable, upgradeable, multipurpose facility that will deliver orders of magnitude increases in sensitivity and mapping speeds over current and planned submillimeter observatories.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.01438-b31b1b.svg)](https://arxiv.org/abs/2407.01438) | **Instrument Overview of Taurus: A Balloon-borne CMB and Dust Polarization Experiment**  |
|| J. L. May, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2024-07-03*|
|*Comments*| **|
|**Abstract**|            Taurus is a balloon-borne cosmic microwave background (CMB) experiment optimized to map the E-mode polarization and Galactic foregrounds at the largest angular scales ($\ell$ $\lt$ 30) and improve measurements of the optical depth to reionization ($\tau$). This will pave the way for improved measurements of the sum of neutrino masses in combination with high-resolution CMB data while also testing the $\Lambda CDM$ model on large angular scales and providing high-frequency maps of polarized dust foregrounds to the CMB community. These measurements take advantage of the low-loading environment found in the stratosphere and are enabled by NASA's super-pressure balloon platform, which provides access to 70% of the sky with a launch from Wanaka, New Zealand. Here we describe a general overview of Taurus, with an emphasis on the instrument design. Taurus will employ more than 10,000 100 mK transition edge sensor bolometers distributed across two low-frequency (150, 220 GHz) and one high-frequency (280, 350 GHz) dichroic receivers. The liquid helium cryostat housing the detectors and optics is supported by a lightweight gondola. The payload is designed to meet the challenges in mass, power, and thermal control posed by the super-pressure platform. The instrument and scan strategy are optimized for rigorous control of instrumental systematics, enabling high-fidelity linear polarization measurements on the largest angular scales.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2407.00153/./figure4.png', 'tmp_2407.00153/./figure5.png', 'tmp_2407.00153/./figure2.png', 'tmp_2407.00153/./figure1.png']
copying  tmp_2407.00153/./figure4.png to _build/html/
copying  tmp_2407.00153/./figure5.png to _build/html/
copying  tmp_2407.00153/./figure2.png to _build/html/
copying  tmp_2407.00153/./figure1.png to _build/html/
exported in  _build/html/2407.00153.md
    + _build/html/tmp_2407.00153/./figure4.png
    + _build/html/tmp_2407.00153/./figure5.png
    + _build/html/tmp_2407.00153/./figure2.png
    + _build/html/tmp_2407.00153/./figure1.png
found figures ['tmp_2407.00166/./Figures/Integrated_exsitu_mass_merger_rate.png', 'tmp_2407.00166/./Figures/Integrated_exsitu_mass_Rot_type_original_std.png', 'tmp_2407.00166/./Figures/ExsituF_vs_total_mass_mangia_manga_tng50.png', 'tmp_2407.00166/./Figures/model_uncertainties.png']
copying  tmp_2407.00166/./Figures/Integrated_exsitu_mass_merger_rate.png to _build/html/
copying  tmp_2407.00166/./Figures/Integra

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# MIDIS. Near-infrared rest-frame morphology   of massive galaxies\ at $3<z<5.5$ in the Hubble eXtreme Deep Field

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.00153-b31b1b.svg)](https://arxiv.org/abs/2407.00153)<mark>Appeared on: 2024-07-03</mark> -  _Submitted to A&A. 7 pages, 4 figures. Comments welcome_

</div>
<div id="authors">

L. Costantin, et al. -- incl., <mark>F. Walter</mark>, <mark>T. K. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Thanks to decades of observations   using the Hubble Space Telescope (HST), the structure   of galaxies at redshift $z>2$ has been widely studied   in the rest-frame ultraviolet regime, which traces recent star formation   from young stellar populations. But, we still have little information   about the spatial distribution of the older, more evolved, stellar populations,   constrained by the rest-frame infrared portion of galaxies'   spectral energy distribution. We present the morphological characterization of a sample   of 21 massive galaxies ( $\log(M_{\star}/M_{\odot})>9.5$ ) at redshift $3<z<5.5$ .   These galaxies are observed as part of the Guaranteed Time Observations   program MIDIS with the Mid-Infrared Instrument (MIRI) onboard the James Webb Space Telescope (JWST).   The deep MIRI 5.6 $\mu$ m imaging (28.64 mag $5\sigma$ depth), allows us to   characterize for the first time the rest-frame near-infrared structure of galaxies   beyond cosmic noon, at higher redshifts than possible with NIRCam, tracing their older stellar populations. We derive the galaxies' non-parametric morphology,   focusing on the Gini, $M_{20}$ , concentration, and deviation statistics.   Furthermore, we model the galaxies' light distribution with a single Sérsic   component and derive their parametric morphology. We find that at $z>3$ massive galaxies show a smooth distribution   of their rest-infrared light, strongly supporting the   increasing number of regular disk galaxies   already in place at early epochs.   On the contrary, the ultraviolet structure obtained from HST   observations (WFC3/F160W) is generally more irregular,   catching the most recent episodes of star formation.   Importantly, we find a segregation of morphologies across cosmic time,   having massive galaxies at redshift $z>4$ later-type morphologies compared to $z\sim3$ galaxies.   These findings suggest a transition phase in galaxy assembly and central mass build up   already taking place at $z\sim3-4$ . MIRI provides unique information about   the structure of the mature stellar population of high-redshift galaxies,   allowing us to unveil that massive galaxies beyond cosmic noon   are prevalently compact disk galaxies with smooth mass distribution.

</div>

<div id="div_fig1">

<img src="tmp_2407.00153/./figure4.png" alt="Fig4.1" width="50%"/><img src="tmp_2407.00153/./figure5.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Gini-$M_{20}$ diagram, color-coded according
to concentration (top left panel), deviation (top right panel),
effective radius (bottom left panel), and Sérsic index (bottom right panel).
Galaxies are separated in two redshift bins:
$3 < z < 4$(circles) and $4 < z < 5.5$(squares).
 (*fig:figure4*)

</div>
<div id="div_fig2">

<img src="tmp_2407.00153/./figure2.png" alt="Fig1" width="100%"/>

**Figure 1. -** Gini-$M_{20}$ diagram.
Orange stars correspond to non-parametric morphology
measured on MIRI F560W images. The green points and shaded region
correspond to non-parametric morphology
measured on WFC3 F160W images.
Green and light red contours represent the $2\sigma$
levels at (observed) 1.6 $\mu$m and 5.6 $\mu$m, respectively.
ETGs, LTGs, and mergers are separated according to Eqs. (4) in \citet{Lotz.J:2008}.
 (*fig:figure2*)

</div>
<div id="div_fig3">

<img src="tmp_2407.00153/./figure1.png" alt="Fig3" width="100%"/>

**Figure 3. -** Near-infrared rest-frame morphology
(MIRI F560W band) of the 21 galaxies studied in this work.
The cutouts are $35 \times 35$ pixel$^2$,
which corresponds to $2.1 \times 2.1$ arcsec$^2$.
We report the angular resolution as the FWHM of the PSF.
 (*fig:figure1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.00153"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\rev}[1]{\textcolor{red}{#1}}$
$\newcommand{\apj}{$
$  Astrophysical Journal$
$}$
$\newcommand{\apjl}{$
$  Astrophysical Journal, Letters$
$}$
$\newcommand{\apjs}{$
$  Astrophysical Journal, Supplement$
$}$
$\newcommand{\aj}{$
$  Astronomical Journal$
$}$
$\newcommand{\araa}{$
$  Annual Review of Astron and Astrophysis$
$}$
$\newcommand{\mnras}{$
$  Monthly Notices of the RAS$
$}$
$\newcommand{\aap}{$
$  Astronomy and Astrophysics$
$}$
$\newcommand{\pasp}{$
$  Publications of the ASP$
$}$
$\newcommand{\rmxaa}{$
$  Revista Mexicana de Astronomia y Astrofisica$
$}$
$\newcommand{\na}{$
$  New Astronomy$
$}$</div>



<div id="title">

# Constraints on the in-situ and ex-situ stellar masses in nearby galaxies with Artificial Intelligence

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.00166-b31b1b.svg)](https://arxiv.org/abs/2407.00166)<mark>Appeared on: 2024-07-03</mark> -  _16 pages, 7 Figures. Accepted for publication_

</div>
<div id="authors">

E. Angeloudi, et al. -- incl., <mark>L. Eisert</mark>, <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** The hierarchical model of galaxy evolution suggests that the impact of mergers is substantial on the intricate processes that drive stellar assembly within a galaxy. However, accurately measuring the contribution of accretion to a galaxy's total stellar mass and its balance with in-situ star formation poses a persistent challenge, as it is neither directly observable nor easily inferred from observational properties. Here, we present theory-motivated predictions for the fraction of stellar mass originating from mergers in a statistically significant sample of nearby galaxies, using data from MaNGA. Employing a robust machine learning model trained on mock MaNGA analogs (MaNGIA) in turn obtained from a cosmological simulation (TNG50), we unveil that in-situ stellar mass dominates almost across the entire stellar mass spectrum ( $10^9M_\odot<M_\star<10^{12}M_\odot$ ). Only in more massive galaxies ( $M_\star>10^{11}M_\odot$ ) does accreted mass become a substantial contributor, reaching up to 35-40 \% of the total stellar mass. Notably, the ex-situ stellar mass in the nearby universe exhibits significant dependence on galaxy characteristics, with higher accreted fractions favored by elliptical, quenched galaxies and slow rotators, as well as galaxies at the center of more massive dark matter halos.

</div>

<div id="div_fig1">

<img src="tmp_2407.00166/./Figures/Integrated_exsitu_mass_merger_rate.png" alt="Fig1" width="100%"/>

**Figure 1. -** The contribution of the in-situ vs. the ex-situ stellar mass component to the total stellar mass for $\sim$ 10,000 MaNGA galaxies as it is predicted from the Neural Network model trained on the MaNGIA mock dataset. The mean total stellar mass at a given stellar mass bin is drawn with a black solid line and should roughly correspond to a straight line. The average  contribution of the in-situ stellar mass created through internal star formation at each stellar mass bin is drawn with a blue solid line. Correspondingly, the red solid line shows the average contribution originating from accretion/mergers (ex-situ). The error bars contain the 68 per cent of all data. For comparison, we plot the mass of the stellar halo (to distinguish from dark matter halo mass) for Milky Way2020ARA&A..58..205H and M312017MNRAS.466.1491H with orange stars (a proxy for accreted mass) and a rough projection of ex-situ stellar mass for Milky Way after it mergers with LMC with a green star (details in discussion). The open circles correspond to an independent ex-situ stellar mass measurement for two early-type galaxies, NGC 1380 and NGC 1427, members of the Fornax galaxy cluster, using population-orbital decomposition methods in combination with cosmological simulation TNG50 from ref.2022A&A...664A.115Z The red shaded region corresponds to an independent measurement of mass from major mergers for massive galaxies from ref. 2022ApJ...940..168C It is apparent that the in-situ stellar mass dominates across the stellar mass range. The ex-situ stellar mass contribution becomes significant only for massive galaxies, revealing a strong dependency of the level of accretion to the stellar mass. Our measurements are roughly compatible with completely independent measurements in the literature. (*fig:manga_recovery*)

</div>
<div id="div_fig2">

<img src="tmp_2407.00166/./Figures/Integrated_exsitu_mass_Rot_type_original_std.png" alt="Fig6" width="100%"/>

**Figure 6. -** Dependencies of the ex-situ stellar mass on rotation. For comparison to the global relation, the mean stellar mass originating from accretion/mergers across the stellar mass range is drawn with a solid black line. At a fixed stellar mass, slow rotators (red dashed line) exhibit higher ex-situ stellar mass than their fast rotator counterparts (blue dashed line). The error bars contain the 68 per cent of the data. The star points correspond to outliers, galaxies classified as fast rotators with a predicted fraction of ex-situ stellar mass > 0.5. We plot 6 SDSS images of the outliers' sample and identify several intriguing actual fast rotators (left-hand side images) as well as galaxies undergoing a merger event (right-hand side images). The image labels correspond to the MaNGA ID of each galaxy. (*fig:exsitu_rotation*)

</div>
<div id="div_fig3">

<img src="tmp_2407.00166/./Figures/ExsituF_vs_total_mass_mangia_manga_tng50.png" alt="Fig2.1" width="50%"/><img src="tmp_2407.00166/./Figures/model_uncertainties.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Validation of the NN model and its uncertainties on the MaNGIA mock dataset and the MaNGA real galaxies. (a) The recovery of the median ex-situ stellar mass fraction vs. stellar mass relation for $\sim$ 10,000 MaNGA galaxies (red solid line) as it is predicted from the ensemble of the NN models trained on the MaNGIA mock dataset. For a visual validation of the model, both the ground-truth (grey dashed line) and the model predictions (blue dash-dotted line) are shown for the ex-situ fractions of the MaNGIA test set, revealing a close recovery of the underlying relation. The median relation for the TNG50 simulation, from which the MaNGIA sample originates, at redshift $z = 0$ is shown with a green dotted line. The shaded regions and error bars contain the 68 per cent of all data. (b) The coverage statistical test applied on the MaNGIA test set. The black solid line corresponds to the median of the uncertainty calibration of the models trained on MaNGIA. The perfect calibration for the uncertainties is shown with a diagonal dashed blue line. Our models seem to follow closely the perfect calibration, which is the desired behavior. On the top, we also show the distribution of the produced ensembled uncertainties of the models on MaNGIA and MaNGA and find that they are comparable, with the model producing slightly higher uncertainties on the unknown MaNGA dataset. (*fig:uncertainties*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.00166"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\MSUN}{{\rm M}_{\sun}}$
$\newcommand{\MHOST}{M_{\rm 200c, host}}$
$\newcommand{\RHOST}{r_{\rm 200c, host}}$
$\newcommand{\RSB}{r_{\rm *,SB=26.5}}$
$\newcommand{\RHALF}{r_{\rm *,1/2}}$
$\newcommand{\RHALFV}{r_{\rm V,1/2}}$
$\newcommand{\RHALFGEN}[1]{r_{\rm *,1/2,#1}}$
$\newcommand{\MSTAR}{M_{\rm *}}$
$\newcommand{\MGEN}[1]{M_{\rm #1}}$
$\newcommand{\RGEN}[1]{R_{\rm #1}}$
$\newcommand{\XXC}{\textbf{SM2}}$
$\newcommand{\XVC}{\textbf{SM1}}$
$\newcommand{\XC}{\textbf{FM}}$
$\newcommand{\XCX}{\textbf{LM1}}$
$\newcommand{\XCXX}{\textbf{LM2}}$
$\newcommand{\XCRII}{\textbf{FM-R2}}$
$\newcommand{\XCRIII}{\textbf{FM-R3}}$
$\newcommand{\targetMerger}{z \approx 2}$
$\newcommand{\vgn}{VG}$
$\newcommand{\tng}{TNG}$
$\newcommand{\gjNotes}[1]{{\color{Green} #1}}$
$\newcommand{\gjChanges}[1]{{\color{Black} #1}}$
$\newcommand{\apChanges}[1]{{\color{Blue} #1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The PARADIGM project I: How early merger histories shape the present-day sizes of Milky-Way-mass galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.00171-b31b1b.svg)](https://arxiv.org/abs/2407.00171)<mark>Appeared on: 2024-07-03</mark> -  _Submitted to MNRAS. 22 pages. Comments are welcome_

</div>
<div id="authors">

G. D. Joshi, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** The way in which mergers affect galaxy formation depends on both feedback processes, and on the geometry and strength of the mergers themselves. We introduce the PARADIGM project, where we study the response of a simulated Milky-Way-mass galaxy forming in a cosmological setting to differing merger histories, using genetically modified initial conditions. Each initial condition is simulated with the VINTERGATAN and IllustrisTNG codes. While VINTERGATAN has been developed with an emphasis on resolving the cold interstellar medium, IllustrisTNG uses a subgrid two-phase model and consequently scales to large volume simulations, making them ideal to examine complementary views on how merger histories and feedback interact. Our genetic modifications alter the mass ratio of an important $\targetMerger$ merger while maintaining the halo's $z=0$ mass. Whether simulated with VINTERGATAN or IllustrisTNG, smaller mass ratios for this early merger result in larger galaxies at $z=0$ , due to a greater build up of a kinematically cold disc. We conclude that such broad trends are robustly reproducible; however, the normalization of the resulting stellar sizes is substantially different in the two codes (ranging between $0.5-1.7 \rm{kpc}$ for VINTERGATAN but $1.3-7.0 \rm{kpc}$ for IllustrisTNG). The VINTERGATAN galaxies systematically form stars earlier, leading to a larger bulge component. Despite the difference in size normalization, both simulation suites lie on the observed size-mass relation for their respective morphological types. In light of these results, we discuss the interplay between internal processes and large scale gravitational interactions and gas accretion, and how the two galaxy models converge on similar emergent trends but along different evolutionary pathways.

</div>

<div id="div_fig1">

<img src="tmp_2407.00171/./figures/half_light_radius_evolution.png" alt="Fig6.1" width="50%"/><img src="tmp_2407.00171/./figures/half_mass_radius_evolution.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Evolution of 2D half-light (top) and 3D half-mass (bottom) radii for the $\tng$(left) and $\vgn$(right) sets of simulations. Each radius has been normalised by the $z=0$ radius of the fiducial run from the corresponding set. Bottom panels show the evolution of stellar mass within $3 $\RHALF$$. The dotted and dashed blue lines in the left panels show the re-simulations of the fiducial ICs with the $\tng$  model to show stochastic scatter ($\XC$RII  and $\XC$RIII  respectively). The $\tng$  galaxies show a clear trend of more rapid size growth as a function of time for the smaller merger scenarios, for both half-light and half-mass radii, beginning at $z \sim 1$. Such a trend is also seen for the $\vgn$  simulations when considering half-light radii; a trend is not obvious for the half-mass radii however, likely due to their larger bulge fractions (see Section \ref{sec:kinComp}). The rapid rise in half-light radius for the $\vgn$$\XXC$  simulation at $z \sim 0.2$ is caused by an influx of gas resulting in SF in the outskirts of the galaxy. We have restricted the y-axis values to show the majority of the trends in all simulations more clearly and therefore do not show the peak of this feature, which is approximately at $\RHALF/$\RHALF$GEN{\rm z=0,fid.} = 5.9$ and $z=0.2$. (*fig:sizeEvolution*)

</div>
<div id="div_fig2">

<img src="tmp_2407.00171/./figures/circ_mass_evolution_stars_10R.png" alt="Fig9" width="100%"/>

**Figure 9. -** Evolution of the stellar mass in disc (red), bulge (gray) and counter-rotating disc (orange) components within $10 $\RHALF$$ as a function of cosmic time, for the $\tng$(top row) and $\vgn$(bottom row) simulations. The masses have been normalised by the galaxies' $z=0$ stellar mass within $10 $\RHALF$$. We omit the two intermediate runs for clarity. The two additional runs of the $\tng$  fiducial simulations ($\XC$RII  and $\XC$RIII) are shown with dotted and dashed curves in the top middle panel. Dotted black lines indicate 10 and 100 percent of $M_{\rm tot,z=0}$ for reference. In the $\tng$  simulations, the increase in $\targetMerger$  merger mass ratio results in a proportionally higher build-up of bulge mass compared to disc mass. In the $\vgn$  simulations, although the trends are not as clear. Additionally in the $\tng$$\XC$XX  case, we also see the rapid build-up of the counter-rotating disc component at $z \sim 0.5$, which then accounts for nearly 10 per cent of the galaxy mass. The simultaneous decrease in disc mass is not accompanied by a loss of total stellar mass, but rather due to the transfer of disc stars to the bulge component. In the $\vgn$$\XC$XX  case, we find a similar relative increase in bulge mass, resulting in an equal proportion of disc and bulge mass at $z=0$. (*fig:circCompEvolution_allStars*)

</div>
<div id="div_fig3">

<img src="tmp_2407.00171/./figures/circularity_parameter_distribution_at_z0_within_10rhalf.png" alt="Fig8" width="100%"/>

**Figure 8. -** Mass-weighted probability distribution functions of the circularity parameter $\epsilon$ for all stellar particles within $10 $\RHALF$$ at $z=0$. Both sets of simulations show a prominent disc component at $\epsilon \approx 0.9$ and a secondary bulge component at $\epsilon=0$, except for the $\XC$XX  cases which are bulge-dominated galaxies. Furthermore, the smaller merger scenarios result in a relatively more massive disc component. In the $\tng$  simulations, the disc component is proportionally more important and corresponds to a kinematically colder disc than in the $\vgn$  simulations. Two other important features are evident in the distributions: a `counter-rotating disc' in the $\tng$$\XC$XX  simulation, and a `disturbed disc' in the $\vgn$$\XC$X  simulation. (*fig:circDistributionZ0*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.00171"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\HL}[1]{{\color{red}  #1}}$</div>



<div id="title">

# Direct observational evidence of multi-epoch massive star formation in G24.47+0.49

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.01063-b31b1b.svg)](https://arxiv.org/abs/2407.01063)<mark>Appeared on: 2024-07-03</mark> -  _18 pages, 7 figures, accepted for publication in The Astrophysical Journal Letters_

</div>
<div id="authors">

A. Saha, et al. -- incl., <mark>S. Li</mark>

</div>
<div id="abstract">

**Abstract:** Using new continuum and molecular line data from the ALMA Three-millimeter Observations of Massive Star-forming Regions (ATOMS) survey and archival VLA, 4.86 GHz data, we present direct observational evidence of hierarchical triggering relating three epochs of massive star formation in a ring-like H ${\small II}$ region, G24.47+0.49. We find from radio flux analysis that it is excited by a massive star(s) of spectral type O8.5V–O8V from the first epoch of star formation.The swept-up ionized ring structure shows evidence of secondary collapse, and within this ring a burst of massive star formation is observed in different evolutionary phases, which constitutes the second epoch.ATOMS spectral line (e.g., HCO $^+$ (1--0)) observations reveal an outer concentric molecular gas ring expanding at a velocity of $\sim$ 9 $ \rm km s^{-1}$ , constituting the direct and unambiguous detection of an expanding molecular ring.It harbors twelve dense molecular cores with surface mass density greater than 0.05 $ \rm g cm^{-2}$ , a threshold typical of massive star formation. Half of them are found to be subvirial, and thus in gravitational collapse, making them third  epoch of potential massive star-forming sites.

</div>

<div id="div_fig1">

<img src="tmp_2407.01063/./f1-spitzer_vla_3mm_ha_horizontal_w1core_4PANELS_H13CO_v2revisit_v3_R1.png" alt="Fig3" width="100%"/>

**Figure 3. -**  Morphology of the region associated with G24.47. (a) _ Spitzer_-IRAC colour composite image overlaid with VLA 4.86 GHz contours at 3, 7, 20, 50, 90, 110, 120 times $\sigma$(= 0.4 $\rm mJy beam^{-1}$). (b) VLA 4.86 GHz map overlaid with ATOMS H40$\alpha$ contours (Gaussian smoothed over 5 pixels) starting at 2$\sigma$(= 0.04 $\rm Jy beam^{-1} km s^{-1}$) in steps of 1$\sigma$. The displayed ellipses are identified VLA cores with their central positions (X) marked. The insets show the H40$\alpha$ spectra (boxcar smoothed by four channels; velocity resolution of 6.0 $\rm km s^{-1}$) of the cores along with their respective Gaussian fits.  The LSR velocity of 101.5 $\rm km s^{-1}$ is indicated in each. (c) ALMA 3 mm map overlaid with the H40$\alpha$ contours displayed in (b). The displayed ellipses are identified 3 mm cores with their central positions (+) marked. (d) Moment zero map (3-pixel smoothed using Gaussian kernel) of H$^{13}$CO$^+$ in the velocity range 93.0 to 113.0 $\rm km s^{-1}$. The displayed ellipses are identified molecular cores with their central positions (+) marked. Ellipses drawn in the lower left corner of (b), (c), and (d) represent the beams of the respective maps displayed. (*fig:RGB_vla_ha*)

</div>
<div id="div_fig2">

<img src="tmp_2407.01063/./f2-all_composite_moment0_1_2compo_full_with_Ha_NOLINESv4.png" alt="Fig4" width="100%"/>

**Figure 4. -** Moment zero (left three) and moment one (rightmost) maps of HCO$^+$ and H40$\alpha$ observed towards G24.47 are shown in panels (a) -- (d) and (e) -- (h), respectively. The velocity ranges used to obtain the moment zero maps are given in the top left of each panel. The colour bar indicates the flux scale in $\rm Jy beam^{-1} km s^{-1}$ and $\rm km s^{-1}$ for moment zero and moment one maps, respectively.
    The overlaid contours (in panels (a) -- (d)) show the H40$\alpha$ emission (presented as colorscale in panel (e)) with contour levels starting at 2$\sigma$( $\sigma =$ 0.04 $\rm Jy beam^{-1} km s^{-1}$) in steps of 1$\sigma$.
    These contours are smoothed over five pixels using Gaussian kernel. The moment zero maps (in panels (e) -- (g)) are smoothed across three pixels using Gaussian kernel.
    The beam is indicated at the bottom left corner in each panel. (*fig:moment-maps*)

</div>
<div id="div_fig3">

<img src="tmp_2407.01063/./f6a-appendix-Tex_map_molcore_ellipse.png" alt="Fig2.1" width="50%"/><img src="tmp_2407.01063/./f6b-appendix-cdmap_molcores_annotate_v2revisit.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** (a) Effective line of sight excitation temperature map generated using HCO$^+$. (b) Column density map towards G24.47, generated using molecular transitions H$^{13}$CO$^+$ and HCO$^+$. The ellipses represent the apertures of identified molecular cores. In both panels, the overlaid contours show the H40$\alpha$ emission with contour levels starting at 2$\sigma$( $\sigma =$ 0.04 $\rm Jy beam^{-1} km s^{-1}$) in steps of 1$\sigma$. These contours are smoothed over 5 pixels using Gaussian kernel.  The beam size of 2.5$\arcsec$ is indicated at the bottom left.  (*fig:cdmap_molcore*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.01063"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

110  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
